In [1]:
from __future__ import (print_function, absolute_import, unicode_literals)

import numpy as np
import keras
import keras.backend as K
from keras import datasets

from models import CNN, VGG8
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import (
    ReduceLROnPlateau,
    CSVLogger,
    EarlyStopping,
    ModelCheckpoint)

import tensorflow as tf

from pulearn.utils import synthesize_pu_labels

global _SESSION
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
_SESSION = tf.Session(config=config)
K.set_session(_SESSION)

Using TensorFlow backend.


# CIFAR10

In [2]:
num_classes = 10
batch_size = 64
epochs = 50
data_augmentation = True
checkpoint = None
# checkpoint = 'model_checkpoint_cifar10_wide_resnet.h5'
title = 'cifar10_vgg8'

In [3]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

x_train = x_train.astype(K.floatx())
x_test = x_test.astype(K.floatx())

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Untaring file...


In [4]:
pct_missing = 0.5
y_train_pu = synthesize_pu_labels(y_train, random_state=42, verbose=True)

('Positive (pct_missing=0.0):', 225000, ' vs.', 225000)
('Positive (pct_missing=0.1):', 225000, ' vs.', 202406)
('Positive (pct_missing=0.2):', 225000, ' vs.', 180014)
('Positive (pct_missing=0.3):', 225000, ' vs.', 157742)
('Positive (pct_missing=0.4):', 225000, ' vs.', 135367)
('Positive (pct_missing=0.5):', 225000, ' vs.', 112793)
('Positive (pct_missing=0.6):', 225000, ' vs.', 89651)
('Positive (pct_missing=0.7):', 225000, ' vs.', 66951)
('Positive (pct_missing=0.8):', 225000, ' vs.', 44603)
('Positive (pct_missing=0.9):', 225000, ' vs.', 22458)
('Positive (pct_missing=1.0):', 225000, ' vs.', 0)


In [5]:
if title == 'cifar10_vgg8':
    model = VGG8(input_shape=x_train.shape[1:], num_classes=num_classes)
elif title == 'cifar10_wide_resnet':
    model = WideResidualNetwork(depth=28, width=8, dropout_rate=0.7,
                                classes=num_classes, include_top=True,
                                weights=None)
if checkpoint is not None:
    model = load_model(checkpoint)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()


checkpointer = ModelCheckpoint(
    filepath="model_checkpoint_{}_pu_{}.h5".format(title, pct_missing),
    verbose=1,
    save_best_only=True)


csv_logger = CSVLogger(
    'csv_logger_{}_pu_{}.csv'.format(title, pct_missing))


early_stopper = EarlyStopping(monitor='val_loss',
                              min_delta=0.001,
                              patience=20)


lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=20,
                               min_lr=0.5e-6)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
__________

In [ ]:
def normalize(x):
    """Substract mean and Divide by std."""
    x -= np.array([125.3, 123.0, 113.9], dtype=K.floatx())
    x /= np.array([63.0, 62.1, 66.7], dtype=K.floatx())
    return x


x_train = normalize(x_train)
x_test = normalize(x_test)

if not data_augmentation:
    print('No data augmentation applied.')
    model.fit(x_train, y_train_pu[pct_missing],
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=[csv_logger, checkpointer, early_stopper]) 
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        # randomly rotate images in the range (degrees, 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train_pu[pct_missing],
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        callbacks=[csv_logger, checkpointer, early_stopper])
    model.save('{}_pu_{}.h5'.format(title, pct_missing))

Using real-time data augmentation.
Epoch 1/50
